
TOPIC 3: 
Implement the “Self Alignment with Instruction Backtranslation” paper. When fine tuning the model, use LoRA. You will not be able to do full finetuning because there is not enough memory.

Link to paper: https://arxiv.org/pdf/2308.06259.pdf
Colab’s GPU usage is limited. Try to first prototype and get things working on the CPU first before training on the GPU with the full dataset. If you are not able to connect to a GPU on colab, you can try to create a PyTorch Lightning Studio or a Kaggle notebook.

In particular:
Finetune the base language model (llama2 7B) with (output, instruction) pairs {(yi, xi)} from the seed data to obtain a backward model Myx := p(x|y). In other words, finetune a model that uses the output to predict the instruction. Use the openassistant-guanaco training set dataset. (25 points)
Push the backwards model to HF and paste url here
Self-Augmentation -- Randomly sample a subset of size 150 and generate instructions from the LIMA dataset’s completions and filtering out any mutli-turn examples. Print out 5 examples of generated instructions. (25 points)
(generated instructions from backwards model, response is from LIMA) pairs
Single turn: 
Single turn: (What is the capital of France?, Paris)
Multi turn: (What is the meaning of life, 42, Why is it 42?, That’s universe, ...)
Self curation (selecting high quality examples) using few shot prompting in addition to the prompt in Table 1 of the paper. Print out 5 examples of high quality examples and 5 examples of low quality examples. (25 points)
Push the dataset to HF hub and paste the url here
Goal is to filter out bad samples
Method: using an LLM to rate the example
LLM (meta/llama-7b-chat-hf): LLM(“Evaluate the quality of the instruction/response pair” + example.” Rate it from 1-5)
Finetune base model on dataset generated by step 3. Print out 5 example responses. (25 points)
Push the instruction fine tuned model to HF hub and paste the url here



In [ ]:
!nvidia-smi

Wed Nov 13 00:02:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install -q datasets

In [ ]:
import bitsandbytes
import transformers

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn as nn
from torch.nn import functional as F
from torch import optim

In [ ]:
from transformers import AutoTokenizer
model_tag = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_tag)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from datasets import load_dataset
dataset = load_dataset('timdettmers/openassistant-guanaco')
dataset = dataset['train'].train_test_split(test_size=0.1)
dataset = dataset.filter(lambda x: len(tokenizer.tokenize(x['text'])) < 256)

print(dataset)

alpaca_prompt = """Below is a response that appropriately completes the request. Write an instruction that describes a task.
### Response:
{}

### Instruction:
{}"""

EOS_TOKEN = tokenizer.eos_token

Repo card metadata block was not found. Setting CardData to empty.


Filter:   0%|          | 0/8861 [00:00<?, ? examples/s]

Filter:   0%|          | 0/985 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3011
    })
    test: Dataset({
        features: ['text'],
        num_rows: 356
    })
})


In [ ]:
alpaca_prompt_M0 = """Below is an instruction that contains a task. Write a response that appropriately completes the request.
### Instruction:
{}

### Response:
{}"""


In [ ]:
splits = ['train', 'test']
texts = []
texts_M0=[]
for split in splits:
    for text in dataset[split]["text"]:
        chunks = text.split("### ")
        chunks = [c for c in chunks if len(c) > 0]
        chunks = chunks[:2]
        if not (chunks[0].startswith('Human: ') or chunks[1].startswith('Assistant: ')):
            continue
        instruction = chunks[0].replace("Human: ", "")
        response = chunks[1].replace("Assistant: ", "")

        texts.append(alpaca_prompt.format(response,instruction) + EOS_TOKEN)
        texts_M0.append(alpaca_prompt_M0.format(instruction,response) + EOS_TOKEN)


In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict({"text": texts})
dataset_M0= Dataset.from_dict({"text": texts_M0})

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 3367
})

In [ ]:
dataset_M0["text"][0]

'Below is an instruction that contains a task. Write a response that appropriately completes the request. \n### Instruction:\n¿Cual es el sentido de la vida?\n\n### Response:\nLa pregunta "¿Cuál es el sentido de la vida?" ha sido objeto de debate y reflexión a lo largo de la historia de la humanidad. Aunque no hay una respuesta universalmente aceptada, se pueden encontrar diferentes perspectivas y respuestas según las creencias, valores y experiencias de cada individuo.\n\nAlgunas personas encuentran el sentido de la vida en su fe religiosa o espiritual, mientras que otras pueden encontrarlo en el amor, la familia, el trabajo, la creatividad, la ayuda a los demás o en la búsqueda del conocimiento y la sabiduría.\n\nEn última instancia, el sentido de la vida es una cuestión personal y subjetiva que cada uno debe buscar y descubrir por sí mismo. A través de la exploración de nuestros valores, intereses y metas, podemos encontrar un propósito y una dirección que nos permitan vivir una vid

In [ ]:
max_length = 256
def generate_and_tokenize_prompt(prompt):
    result = tokenizer(
        prompt['text'],
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
tokenized_dataset = dataset.map(generate_and_tokenize_prompt)
tokenized_dataset_M0=dataset_M0.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/3367 [00:00<?, ? examples/s]

Map:   0%|          | 0/3367 [00:00<?, ? examples/s]

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)


In [ ]:
from peft import LoraConfig
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
#pip install -U bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
    model_tag,
    quantization_config=quant_config,
    device_map="auto",
)
model.config.use_cache = False
model.config.pretraining_tp = 1

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "fc1",
        "fc2",
        "dense",
        "lm_head"
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [ ]:
#print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    lr_scheduler_type='cosine',
    max_steps=50,
    learning_rate=2e-5,
    optim="paged_adamw_8bit",
    logging_steps=5,
)

# Trainer setup
trainer = Trainer(
    model=model,  # Wrapped model with adapters
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# Train the model
trainer.train()


max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
5,3.101700
10,2.792300
15,2.139300
20,2.765700
25,2.417200
30,2.591300
35,2.217300
40,2.551100
45,2.462400
50,2.139700


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=2.517799777984619, metrics={'train_runtime': 393.3968, 'train_samples_per_second': 0.127, 'train_steps_per_second': 0.127, 'total_flos': 509465434521600.0, 'train_loss': 2.517799777984619, 'epoch': 0.01485001485001485})

In [ ]:


training_args = TrainingArguments(
    output_dir="./output_1",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    lr_scheduler_type='cosine',
    max_steps=50,
    learning_rate=2e-5,
    optim="paged_adamw_8bit",
    logging_steps=5,
)
trainer_M0 = Trainer(
    model=model,
    train_dataset=tokenized_dataset_M0,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer_M0.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
5,2.623600
10,2.355400
15,1.882000
20,2.192800
25,1.980000
30,2.150400
35,2.014100
40,2.217900
45,2.041200
50,1.790100


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=2.12475528717041, metrics={'train_runtime': 39.7537, 'train_samples_per_second': 1.258, 'train_steps_per_second': 1.258, 'total_flos': 509465434521600.0, 'train_loss': 2.12475528717041, 'epoch': 0.01485001485001485})

In [ ]:
from huggingface_hub import notebook_login

# Use the notebook_login function to log in
notebook_login()

from huggingface_hub import login
login(token="add_token")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
new_model_name = "llama-2-7b-chat-guanaco"
new_model_name_fwd= "llama-2-7b-chat-guanaco_fwd"

trainer.tokenizer = tokenizer
trainer_M0.tokenizer = tokenizer
# Save model and tokenizer
trainer.model.save_pretrained(new_model_name)
trainer.tokenizer.save_pretrained(new_model_name)
trainer.model.push_to_hub('SaniyaEmpire/backward_model')
trainer_M0.model.save_pretrained(new_model_name)
trainer_M0.tokenizer.save_pretrained(new_model_name)
trainer_M0.model.push_to_hub('SaniyaEmpire/forward_model')

print(trainer.model)
print(trainer.tokenizer)


adapter_model.safetensors:   0%|          | 0.00/367M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/367M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

In [ ]:
dataset_lima=load_dataset('GAIR/lima')

0000.parquet:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/27.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
dataset_lima = dataset_lima.filter(lambda x: len((x['conversations'])) < 3)

print(dataset_lima)

Filter:   0%|          | 0/1030 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['conversations', 'source'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['conversations', 'source'],
        num_rows: 300
    })
})


In [ ]:
dataset_lima["train"][0]

{'conversations': ['Can brain cells move? By movement I mean long distance migration (preferably within the brain only).',
  'The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Miller, 2002).\nNeuronal stem cells migr

In [ ]:
# Given output
output_text = """Suppose you have a theorem that says "If $X$, then $Y$."  There are two ways to strengthen such a theorem:\n\n* Assume less.  If you can reduce the number of hypotheses, but still prove the same conclusion, then you have proved a more "powerful" result (in the sense that it applies in more situations).\n* Prove more.  If you can keep the same hypotheses, but add more information to the conclusion, then you have also produced a more "powerful" result.\n\nHere is an easy example from Geometry.\n\n  Let $ABCD$ be a (non-square) rectangle.  Then the internal angle bisectors of the vertices intersect at four points $WXYZ$, which are the vertices of a rectangle.\n\n(You need the condition that $ABCD$ is not a square because if it is a square then all four angle bisectors coincide at a single point.)\nHere are a few ways to strengthen the theorem:\n\n* The hypothesis "$ABCD$ is a (non-square) rectangle" can be relaxed to the more general "$ABCD$ is a (non-rhombic) parallelogram".  The conclusion that $WXYZ$ is a rectangle still holds.\n* Alternatively, you can keep the original hypothesis that $ABCD$ is a (non-square) rectangle, and strengthen to the conclusion to say that $WXYZ$ is not just a rectangle, but a square.\n* Having done that, you can then strengthen the conclusion of the theorem even more, by noting that the diagonal of square $WXYZ$ is equal in length to the difference of the lengths of the sides of $ABCD$.\n* Once you know that, you can now strengthen the theorem even more by (finally) removing the hypothesis that $ABCD$ is non-square, and including the  case in which the four angle bisectors coincide at a single point as forming a "degenerate" square with a diagonal of length zero.\n'"""

# Prepare input prompt for the backward model
prompt = f"Generate an instruction for the following output:\n{output_text}\n generated question:"
input_ids = trainer.tokenizer.encode(prompt, return_tensors="pt")

# Generate instruction using the fine-tuned backward model
generated_ids = trainer.model.generate(input_ids, max_length=500)  # Adjust max_length as needed
generated_instruction = trainer.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(generated_instruction.split("generated question:")[-1].replace("\n", ""))


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1935: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


 What is the strength of the theorem "If $X$, then $Y$"?Please select one of the following options:A) The theorem is not powerful.B


In [ ]:
instructions = []
max_conv = 100
i = 0
for conv in dataset_lima["train"]:
    if i > max_conv:
        break
    ques, ans = conv["conversations"][0], conv["conversations"][1]

    # Prepare input prompt for the backward model
    prompt = f"Generate an instruction for the following output:\n{ans}\n generated question:"
    input_ids = trainer.tokenizer.encode(prompt, return_tensors="pt")

    # Generate instruction using the fine-tuned backward model
    generated_ids = trainer.model.generate(input_ids, max_length=2000)  # Adjust max_length as needed
    generated_instruction = trainer.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    #
    instructions.append({"question": ques, "ans": ans, "generated_ques": generated_instruction})

    if i % 10 == 0:
        print(f"Completed {i} predictions")

    i += 1

augmented_example=[]
# Print or use the generated instructions as needed
for instruction in instructions:
    augmented_example.append({'instruction':instruction["generated_ques"],'output':instruction["ans"]})


Completed 0 predictions
Completed 10 predictions
Completed 20 predictions
Completed 30 predictions
Completed 40 predictions
Completed 50 predictions
Completed 60 predictions
Completed 70 predictions
Completed 80 predictions
Completed 90 predictions
Completed 100 predictions


In [ ]:
len(instructions)

101

In [ ]:
new_augmented_example = []
for example in augmented_example:
    new_example = {
        'instruction': example['instruction'].split("generated question:")[-1].replace("\n", ""),
        'output': example['output']
    }
    new_augmented_example.append(new_example)
new_augmented_example[0]['instruction']


# The second part (if exists) is considered as the generated question


' What are the mechanisms that allow brain cells to migrate?'

In [ ]:
# Define the prompting mechanism
prompt = """Below is an instruction from an user and a candidate answer.  Evaluate whether or
not the answer is a good example of how AI Assistant should respond to the user’s
instruction.  Please assign a score using the following 5-point scale:
1:  It means the answer is incomplete, vague, off-topic, controversial, or not
exactly what the user asked for.  For example, some content seems missing, numbered
list does not start from the beginning, the opening sentence repeats user’s question.
Or the response is from another person’s perspective with their personal experience
(e.g.  taken from blog posts), or looks like an answer from a forum.  Or it contains
promotional text, navigation text, or other irrelevant information.
2:  It means the answer addresses most of the asks from the user.  It does not
directly address the user’s question.  For example, it only provides a high-level
methodology instead of the exact solution to user’s question.
3:  It means the answer is helpful but not written by an AI Assistant.  It addresses
all the basic asks from the user.  It is complete and self contained with the
drawback that the response is not written from an AI assistant’s perspective, but
from other people’s perspective.  The content looks like an excerpt from a blog post,
web page, or web search results.  For example, it contains personal experience or
opinion, mentions comments section, or share on social media, etc.
4:  It means the answer is written from an AI assistant’s perspective with a
clear focus of addressing the instruction.  It provide a complete, clear, and
comprehensive response to user’s question or instruction without missing or
irrelevant information.  It is well organized, self-contained, and written in a
helpful tone.  It has minor room for improvement, e.g.  more concise and focused.
5:  It means it is a perfect answer from an AI Assistant.  It has a clear focus on
being a helpful AI Assistant, where the response looks like intentionally written
to address the user’s question or instruction without any irrelevant sentences.  The
answer provides high quality content, demonstrating expert knowledge in the area, is
very well written, logical, easy-to-follow, engaging and insightful.
Please first provide a brief reasoning you used to derive the rating score, and
then write "Score:  <rating>" in the last line.

Instruction: {}\n
Answer: {}

"""




max_iterations = 100
i = 0
curated_set = []
for example in new_augmented_example:
    if i > max_iterations:
        break
    prompt = prompt.format(example["instruction"], example["output"])
    input_ids = trainer_M0.tokenizer.encode(prompt, return_tensors="pt")
    generated_ids = trainer_M0.model.generate(input_ids, max_new_tokens=512)  # Adjust max_length as needed
    score = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    curated_set.append({"generated_question": example["instruction"], "answer": example["output"], "score":score})
    i+=1


In [ ]:
new_scores = []
print(float(curated_set[1]["score"].rsplit("Score:", 1)[-1].strip(" <>")[:1]))
for sample in curated_set:
    try:
        score = float(sample["score"].rsplit("Score:", 1)[-1].strip(" <>")[:1])
        new_scores.append({
        "generated_question": sample["generated_question"],
        "answer_to_question": sample["answer"],
        "score": score
    })
    except ValueError as ve:
        score = 0
        print(f"Error processing score for {sample}: {ve}")




4.0


In [ ]:
high_ranking_score = float(3)
filtered_scores = [score for score in new_scores if score["score"] >= high_ranking_score]
filtered_scores



[{'generated_question': ' What are the mechanisms that allow brain cells to migrate?',
  'answer_to_question': 'The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Miller, 2002).\nNeuronal stem cells migrate over long 

In [ ]:
texts_for_dataset = []
for data in filtered_scores:
    texts_for_dataset.append(alpaca_prompt_M0.format(data['generated_question'],data['answer_to_question']) + EOS_TOKEN)



In [ ]:
dataset_to_push = Dataset.from_dict({"text":texts_for_dataset})
dataset_to_push.push_to_hub('SaniyaEmpire/quality_dataset')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/SaniyaEmpire/quality_dataset/commit/36e3bf9c466bc43f0c994ed188759dcee0b578bc', commit_message='Upload dataset', commit_description='', oid='36e3bf9c466bc43f0c994ed188759dcee0b578bc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SaniyaEmpire/quality_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SaniyaEmpire/quality_dataset'), pr_revision=None, pr_num=None)

In [ ]:
from datasets import load_dataset
dataset = load_dataset('SaniyaEmpire/quality_dataset')
dataset = dataset['train'].train_test_split(test_size=0.1)

print(dataset)
splits = ['train', 'test']
texts = []


for split in splits:
    for text in dataset[split]["text"]:
        texts.append(text + EOS_TOKEN)


dataset = Dataset.from_dict({"text": texts})
max_length = 256


README.md:   0%|          | 0.00/270 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/101 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 90
    })
    test: Dataset({
        features: ['text'],
        num_rows: 11
    })
})


In [ ]:
def generate_and_tokenize_prompt(prompt):
    result = tokenizer(
        prompt['text'],
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(generate_and_tokenize_prompt)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

model_with_new_dataset = AutoModelForCausalLM.from_pretrained(
    model_tag,
    quantization_config=quant_config,
    device_map="auto",
)
model.config.use_cache = False
model.config.pretraining_tp = 1





Map:   0%|          | 0/101 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "fc1",
        "fc2",
        "dense",
        "lm_head"
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model_with_new_dataset = get_peft_model(model_with_new_dataset, config)

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling



training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    lr_scheduler_type='cosine',
    max_steps=50,
    learning_rate=2e-5,
    optim="paged_adamw_8bit",
    logging_steps=5,
)


trainer_new = Trainer(
    model=model_with_new_dataset,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer_new.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
5,1.999800
10,2.021900
15,2.144300
20,1.526700
25,1.903000
30,2.334400
35,1.867000
40,2.270100
45,1.797900
50,1.869200


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=1.973446102142334, metrics={'train_runtime': 40.1035, 'train_samples_per_second': 1.247, 'train_steps_per_second': 1.247, 'total_flos': 509465434521600.0, 'train_loss': 1.973446102142334, 'epoch': 0.49504950495049505})

In [ ]:

trainer_new.model.push_to_hub('SaniyaEmpire/fine_tuned_instruction')

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/367M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SaniyaEmpire/fine_tuned_instruction/commit/611b3707f397a0a3ec64aba1d0837c710e86eedd', commit_message='Upload model', commit_description='', oid='611b3707f397a0a3ec64aba1d0837c710e86eedd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SaniyaEmpire/fine_tuned_instruction', endpoint='https://huggingface.co', repo_type='model', repo_id='SaniyaEmpire/fine_tuned_instruction'), pr_revision=None, pr_num=None)